# Demo 5: Collaborative Filtering and Comedy! 
------
<img src="images/seinfeld.jpg" width="400" height="400">

#### Real Dataset: http://eigentaste.berkeley.edu/dataset/ Dataset 2 
#### Rate Jokes: http://eigentaste.berkeley.edu

## What are we trying to learn from this dataset?

# QUESTION:  Can Collaborative Filtering be used to find which jokes to recommend to our users?


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import pandas
import cassandra
import pyspark
import re
import os
import matplotlib.pyplot as plt
from IPython.display import IFrame
from IPython.display import display, Markdown
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

#### Helper function to have nicer formatting of Spark DataFrames

In [3]:
#Helper for pretty formatting for Spark DataFrames
def showDF(df, limitRows =  10, truncate = False):
    if(truncate):
        pandas.set_option('display.max_colwidth', 100)
    else:
        pandas.set_option('display.max_colwidth', -1)
    pandas.set_option('display.max_rows', limitRows)
    display(df.limit(limitRows).toPandas())
    pandas.reset_option('display.max_rows')

# DataStax Enterprise Analytics
<img src="images/dselogo.png" width="400" height="200">

## Creating Tables and Loading Tables

### Connect to DSE Analytics Cluster

In [4]:
from cassandra.cluster import Cluster

cluster = Cluster(['127.0.01'])
session = cluster.connect()

### Create Demo Keyspace 

In [5]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS accelerate 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

### Set keyspace 

In [6]:
session.set_keyspace('accelerate')

### Create table called jokes. Our PRIMARY will need to be a unique composite key (userid, jokeid). This will result in an even distribution of the data and allow for each row to be unique. Remember we will have to utilize that PRIMARY KEY in our WHERE clause in any of our CQL queries. 

In [7]:
query = "CREATE TABLE IF NOT EXISTS jokes \
                                    (userid int, jokeid int, rating float, \
                                     PRIMARY KEY (userid, jokeid))"
session.execute(query)

### What do these of these 3 columns represent: 

* **Column 1**: User id
* **Column 2**: Joke id
* **Column 3**: Rating of joke (-10.00 - 10.00) 

### Load Jokes dataset from CSV file (jester_ratings3.csv)
* This is a file I created from the *.dat file and I only have 10,000 rows -- dataset has over 1 million rows
<img src="images/laughing.gif" width="300" height="300">

#### Insert all the Joke Rating Data into the DSE table `jokes`

In [8]:
fileName = 'data/jester_ratings3.csv'
input_file = open(fileName, 'r')

for line in input_file:
    jokeRow = line.split(',')
    query = "INSERT INTO jokes (userid, jokeid, rating)"
    
    query = query + "VALUES (%s, %s, %s)"
    
    session.execute(query, (int(jokeRow[0]), int(jokeRow[1]) , float(jokeRow[2]) ))

#### Do a select * on joke_table WHERE userid = x to verify that data was loaded into the table

In [9]:
query = 'SELECT * FROM jokes WHERE userid = 100'
rows = session.execute(query)
for row in rows:
    print (row.userid, row.jokeid, row.rating)

100 5 -0.875
100 7 9.906000137329102
100 8 -0.843999981880188
100 13 8.937999725341797
100 15 -0.968999981880188
100 16 -9.75
100 17 9.593999862670898


## Machine Learning with DSE Analytics and Apache Spark
<img src="images/sparklogo.png" width="150" height="200">

### Finally time for Apache Spark! 

#### Create a spark session that is connected to DSE. From there load each table into a Spark Dataframe and take a count of the number of rows in each.

In [16]:
spark = SparkSession.builder.appName('demo').master("local").getOrCreate()

jokeTable = spark.read.format("org.apache.spark.sql.cassandra").options(table="jokes", keyspace="accelerate").load()

print ("Table Row Count: ")
print (jokeTable.count())

Table Row Count: 
10000


#### CFilter with PySpark requires that the ratings not be double/foat but int

In [14]:
#joke_df = jokeTable.withColumn("rating", jokeTable.rating.cast('int'))

#### Split dataset into training and testing set 

In [17]:
(training, test) = jokeTable.randomSplit([0.8, 0.2])

training_df = training.withColumn("rating", training.rating.cast('int'))
testing_df = test.withColumn("rating", test.rating.cast('int'))

showDF(training_df)

,userid,jokeid,rating
0,3,5,-9
1,3,7,-9
2,3,8,-7
3,3,13,-2
4,3,15,-9
5,3,16,-9
6,3,17,-9
7,3,18,-9
8,3,20,-6
9,3,33,-9


### Setup for CFliter with ALS

https://spark.apache.org/docs/latest/ml-collaborative-filtering.html

In [37]:
als = ALS(maxIter=5, regParam=0.01, userCol="userid", itemCol="jokeid", ratingCol="rating",
          coldStartStrategy="drop")

model = als.fit(training_df)

In [38]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(testing_df)

# Generate top 10 joke recommendations for each user
userRecs = model.recommendForAllUsers(10)

showDF(userRecs)

# Generate top 10 user recommendations for each joke
jokeRecs = model.recommendForAllItems(10)

,userid,recommendations
0,148,"[(34, 35.08817672729492), (21, 34.19489288330078), (75, 33.810821533203125), (44, 31.955120086669922), (22, 31.689788818359375), (81, 29.63794708251953), (83, 29.347204208374023), (46, 28.51775360107422), (69, 28.306196212768555), (26, 27.55040168762207)]"
1,243,"[(138, 33.15718078613281), (71, 31.30754852294922), (108, 31.254772186279297), (115, 22.185895919799805), (29, 18.956073760986328), (146, 18.66478729248047), (102, 18.170202255249023), (109, 16.25419044494629), (100, 15.723347663879395), (58, 15.562417984008789)]"
2,251,"[(140, 9.199028968811035), (131, 6.886463642120361), (134, 6.8544230461120605), (55, 5.513618469238281), (141, 4.701805114746094), (95, 4.5161333084106445), (73, 3.2440133094787598), (120, 3.053881883621216), (136, 2.061122417449951), (121, 1.7894281148910522)]"
3,85,"[(122, 52.7970085144043), (55, 49.56493377685547), (23, 46.22010803222656), (69, 45.096397399902344), (116, 42.8684196472168), (140, 41.300758361816406), (150, 40.065032958984375), (127, 38.06189727783203), (135, 37.141693115234375), (123, 33.308258056640625)]"
4,137,"[(79, 9.588911056518555), (8, 8.9959077835083), (7, 8.994061470031738), (43, 8.694997787475586), (37, 7.821763515472412), (33, 7.328368663787842), (44, 7.179834842681885), (19, 7.006222724914551), (150, 6.709961414337158), (90, 6.506020545959473)]"
5,65,"[(67, 12.088887214660645), (79, 11.402899742126465), (57, 11.215535163879395), (26, 11.186339378356934), (99, 11.089591026306152), (64, 11.010234832763672), (37, 10.994126319885254), (109, 10.851428985595703), (70, 10.816305160522461), (60, 10.763251304626465)]"
6,53,"[(119, 9.582527160644531), (22, 8.826216697692871), (120, 8.013260841369629), (46, 7.997888565063477), (88, 7.4543375968933105), (95, 7.428121089935303), (121, 7.256031036376953), (34, 7.093655586242676), (110, 6.919713973999023), (61, 6.851659297943115)]"
7,133,"[(88, 16.776498794555664), (87, 16.547439575195312), (119, 13.981500625610352), (118, 13.888110160827637), (134, 13.26351547241211), (111, 12.899371147155762), (110, 12.737327575683594), (120, 12.610612869262695), (48, 12.012154579162598), (37, 11.976896286010742)]"
8,155,"[(124, 40.037967681884766), (141, 37.646480560302734), (87, 29.346630096435547), (40, 23.134788513183594), (63, 18.293651580810547), (52, 14.018779754638672), (103, 12.339641571044922), (48, 10.631392478942871), (19, 9.859904289245605), (43, 9.416757583618164)]"
9,108,"[(58, 30.053936004638672), (130, 16.35774040222168), (114, 15.775555610656738), (121, 15.666524887084961), (33, 15.50494384765625), (25, 15.418022155761719), (113, 14.965984344482422), (37, 14.093564987182617), (141, 12.774852752685547), (107, 12.492015838623047)]"


In [39]:
showDF(userRecs.filter(userRecs.userid == 65))

,userid,recommendations
0,65,"[(67, 12.088887214660645), (79, 11.402899742126465), (57, 11.215535163879395), (26, 11.186339378356934), (99, 11.089591026306152), (64, 11.010234832763672), (37, 10.994126319885254), (109, 10.851428985595703), (70, 10.816305160522461), (60, 10.763251304626465)]"


In [29]:
IFrame(src='images/init94.html', width=700, height=200)

In [30]:
IFrame(src='images/init43.html', width=700, height=200)

In [ ]:
session.execute("""drop table jokes""")